### Import the libraries

In [17]:
import cv2
import os
import numpy as np

### Collect HAAR Cascade filters
- haarcascade_frontalface_alt2.xml

In [18]:
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"

### Take image

In [19]:
image = cv2.imread("Stewie.jpg")

In [20]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [21]:
blurred = cv2.GaussianBlur(gray, (27, 27), 0)

In [22]:
(T, thresh) = cv2.threshold(blurred, 235, 255, cv2.THRESH_BINARY)

In [23]:
(T, threshInv) = cv2.threshold(blurred, 235, 255, cv2.THRESH_BINARY_INV)

In [24]:
image.shape

(681, 1024, 3)

In [25]:
foreground = cv2.bitwise_and(image, image, mask = threshInv)

In [26]:
foreground.shape

(681, 1024, 3)

### Create Cascade Classifiers
- Face
- Eyes

In [27]:
faceCascade = cv2.CascadeClassifier(cascPathface)

### Create file to save the recording

In [28]:
video_capture = cv2.VideoCapture(1)

In [29]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('output_TrungHau_CartoonifyFaces.avi', fourcc, 20.0, (int(video_capture.get(3)), int(video_capture.get(4))))

### Detect face with overlay cartoon pic

In [30]:
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=15,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        resized_image = cv2.resize(foreground, (int(w*1.1), int(h*1.1)))

        roi_y = max(0, y - 35)
        roi_x = max(0, x - 25)
        roi = frame[roi_y:roi_y + resized_image.shape[0], roi_x:roi_x + resized_image.shape[1]]

        # Overlay the resized image on the face
        frame[roi_y:roi_y + resized_image.shape[0], roi_x:roi_x + resized_image.shape[1]] = cv2.addWeighted(
            roi, 1, resized_image, 2, 0)
        
    # Display the resulting frame
    cv2.imshow('Face Video', frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
out.release()
cv2.destroyAllWindows()

### Detect face and eyes

In [ ]:
# Collect HAAR Cascade filters
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

# Create Cascade Classifiers
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        # Draw rectangles on the detected faces
        cv2.rectangle(frame, (x, y), (x + w, y + h),(255, 255, 0), 2)
        faceROI = frame[y:y+h,x:x+w]
        # Detect the eyes
        eyes = eyeCascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (0, 0, 255), 4)

        # Display the resulting frame
    cv2.imshow('Face Video', frame)
    cv2.imshow("Face ROI", faceROI)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()